In [1]:
import pandas as pd

In [43]:
df = pd.read_excel("rawdata.xlsx")

In [44]:
df.head()

,date,time,sensor,location,number,activity,position,location.1
0,2024-01-16,10:10:30,0.0,A1,1,placed,inside,A1
1,2024-01-16,10:12:30,1.0,A2,1,picked,inside,A2
2,2024-01-16,10:28:30,1.0,A3,1,picked,Inside,A3
3,2024-01-16,10:38:20,0.0,A4,1,placed,Inside,A4
4,2024-01-16,10:45:20,1.0,A5,1,placed,inside,A5


In [63]:
df['date']

24    2024-01-16
16    2024-01-16
18    2024-01-16
19    2024-01-16
26    2024-01-16
         ...    
149   2024-01-18
150   2024-01-18
151   2024-01-18
152   2024-01-18
153   2024-01-18
Name: date, Length: 175, dtype: datetime64[ns]

In [47]:
df['datetime'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['time'].astype(str))


In [49]:
df['position'] = df['position'].str.lower()

In [50]:
df = df.sort_values(by='datetime')

## Total Duration

In [51]:
df['time_diff'] = df['datetime'].diff()
df['duration'] = df['time_diff'].shift(-1).fillna(pd.Timedelta(seconds=0))

In [52]:
inside_df = df[df['position'] == 'inside']
outside_df = df[df['position'] == 'outside']

In [53]:
inside_duration = inside_df.groupby(df['datetime'].dt.date)['duration'].sum()
outside_duration = outside_df.groupby(df['datetime'].dt.date)['duration'].sum()

In [54]:
result_df = pd.DataFrame({
    'date': inside_duration.index,
    'inside_duration': inside_duration.values,
    'outside_duration': outside_duration.reindex(inside_duration.index, fill_value=pd.Timedelta(seconds=0)).values
})

In [57]:
result_df.head()

,date,inside_duration,outside_duration
0,2024-01-16,1 days 10:08:10,0 days 00:00:00
1,2024-01-17,1 days 00:00:00,0 days 00:00:00
2,2024-01-18,0 days 08:32:57,0 days 00:46:53


## No. of Placing and picking

In [58]:
activity_counts = df.groupby(['date', 'activity']).size().reset_index(name='count')

In [59]:
pivot_table = activity_counts.pivot(index='date', columns='activity', values='count').fillna(0)

In [60]:
pivot_table.columns = ['number_of_picked', 'number_of_placed']

In [61]:
result_df = pivot_table.reset_index()

In [62]:
result_df.head()

,date,number_of_picked,number_of_placed
0,2024-01-16,40,40
1,2024-01-17,10,9
2,2024-01-18,37,39
